<a href="https://colab.research.google.com/github/Reth202/Tesis_Analisis_Del_Sentimiento/blob/main/Entregable_2_Proyecto_final_19_Junio_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
username = "USER_PROYFINAL"
dsn = "dbproyfinal_high"
pw = "UserProyFinal#40_24"
wallet_pw="^&$#7aBcdeFgHiJkLmNpQrStUvWxYz@!%"

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!chmod +x /content/drive/MyDrive/TrabajoFinal/Codigo/ColabVersiones/opensmile-3.0.2-linux-x86_64/bin/SMILExtract
!chmod +r "/content/drive/MyDrive/TrabajoFinal/Codigo/ColabVersiones/opensmile-3.0.2-linux-x86_64/config/is09-13/IS09_emotion.conf"

In [ ]:
path = 'C:\Users\Lenovo\Documents\Maestría\Trabajo Final Tesis\Entregable 3\Tesis_Analisis_Del_Sentimiento\Sentimientos\'
lst = []

os.chdir(path)

for subdir, dirs, files in os.walk(path):
  for file in files:
      try:
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        file = file[6:8]
        arr = mfccs, file
        lst.append(arr)
      except ValueError:
        continue

In [ ]:
# Función para extraer características con OpenSMILE
def extract_features(audio_path, output_path, config_path="C:\Users\Lenovo\Documents\Maestría\Trabajo Final Tesis\Entregable 3\Tesis_Analisis_Del_Sentimiento\opensmile-3.0.2-linux-x86_64\config\is09-13\IS09_emotion.conf"):
    !chmod +r "{config_path}"

    # Construir el comando para ejecutar OpenSMILE
    command = f'"C:\Users\Lenovo\Documents\Maestría\Trabajo Final Tesis\Entregable 3\Tesis_Analisis_Del_Sentimiento\opensmile-3.0.2-linux-x86_64\bin\SMILExtract" -C "{config_path}" -I "{audio_path}" -O "{output_path}"'

    # Ejecutar el comando y capturar el resultado
    result = subprocess.run(command, shell=True, capture_output=True)

    # Manejo de errores si el proceso no termina con éxito
    if result.returncode != 0:
        print(f"Error al ejecutar OpenSMILE:\n{result.stderr.decode('utf-8')}")
        return "Error al ejecutar OpenSMILE"
    else:
        return result

In [ ]:
# Directorio de datos
data_dir = '/content/drive/MyDrive/TrabajoFinal/Dataset_Entrenamiento/'
output_dir = '/content/drive/MyDrive/TrabajoFinal/Dataset_Entrenamiento/features'

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Inicializar una lista para almacenar los datos
data = []

# Recorrer cada carpeta de emociones
for emotion in os.listdir(data_dir):
    emotion_dir = os.path.join(data_dir, emotion)
    if os.path.isdir(emotion_dir):
        # Recorrer cada archivo de audio en la carpeta de emociones
        for audio_file in os.listdir(emotion_dir):
            if audio_file.endswith('.wav'):
                audio_path = os.path.join(emotion_dir, audio_file)
                AudioSinLaExtension = os.path.splitext(audio_file)[0]
                output_path = os.path.join(output_dir, f"{emotion}_{AudioSinLaExtension}.csv")

                # Extraer características
                extract_features(audio_path, output_path)

                # Leer las características extraídas
                with open(output_path, 'r') as file:
                    lines = file.readlines()

                # Encontrar la línea con los datos (después de @data)
                data_start_index = lines.index('@data\n') + 1

                # Extraer los nombres de los atributos y los valores correspondientes
                feature_names = []
                feature_values = []

                for line in lines:
                    if line.startswith('@attribute'):
                        parts = line.split()
                        feature_names.append(parts[1])
                    elif not line.startswith('@') and line.strip():
                        feature_values = line.strip().split(',')


                # Crear un diccionario con las características y sus valores
                feature_data = {feature_names[i]: float(feature_values[i]) if feature_values[i] != '?' else None for i in range(1, len(feature_values))}
                feature_data['label'] = emotion

                # Añadir las características a la lista de datos
                data.append(feature_data)

# Combinar todos los datos en un DataFrame
df = pd.DataFrame(data)

# Guardar los datos en un archivo CSV
df.to_csv('/content/drive/MyDrive/TrabajoFinal/Dataset_Entrenamiento/features/features.csv', index=False)

In [ ]:
# Leer el archivo CSV de características
data = pd.read_csv('/content/drive/MyDrive/TrabajoFinal/Dataset_Entrenamiento/features/features.csv')

# Separar características y etiquetas, excluyendo la columna 'class'
X = data.drop(columns=['label', 'class'])
y = data['label']

# Codificar las etiquetas
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Convertir todas las columnas a numéricas (ignorando errores)
X = X.apply(pd.to_numeric, errors='coerce')

# Eliminar filas con valores faltantes después de la conversión
X = X.dropna()
y_encoded = y_encoded[X.index]  # Ajustar y_encoded para coincidir con el índice de X

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Escalar las características usando StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir el modelo
model_voice = Sequential()
model_voice.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model_voice.add(Dense(64, activation='relu'))
model_voice.add(Dense(len(encoder.classes_), activation='softmax'))  # Asumiendo 3 clases de salida

# Compilar el modelo
model_voice.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model_voice.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model_voice.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

Epoch 1/20
50/50 [==============================] - 3s 14ms/step - loss: 0.5195 - accuracy: 0.8712 - val_loss: 0.0442 - val_accuracy: 0.9900
Epoch 2/20
50/50 [==============================] - 0s 7ms/step - loss: 0.0136 - accuracy: 0.9987 - val_loss: 0.0227 - val_accuracy: 0.9950
Epoch 3/20
50/50 [==============================] - 0s 7ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0192 - val_accuracy: 0.9950
Epoch 4/20
50/50 [==============================] - 0s 6ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0150 - val_accuracy: 0.9975
Epoch 5/20
50/50 [==============================] - 0s 8ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 0.9975
Epoch 6/20
50/50 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0119 - val_accuracy: 0.9975
Epoch 7/20
50/50 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0107 - val_accuracy: 0.9975
Epoch 8/20
50/50 [=

In [ ]:
# Establishing a connection to the Oracle database
try:
    conn = oracledb.connect(
    user= username,
    password= pw,
    dsn= dsn,
    #Colocar ruta del walet en la maquina donde se ejecute, el wallet debe estar descomprimido
    config_dir=r'/content/drive/MyDrive/TrabajoFinal/Codigo/DB/Wallet_DBPROYFINAL',
    wallet_location=r'/content/drive/MyDrive/TrabajoFinal/Codigo/DB/Wallet_DBPROYFINAL',
    wallet_password=wallet_pw
    )
    # Print the version of the database to confirm connection
    print(conn.version)
except Exception as err:
    print("Error en la conexión a la base de datos", err)

cur_01=conn.cursor()


##Cierra la conexion de la base de datos
#conn.close()

def write_db(pfile_name, pupload_date, pfile_size, pclient, pcalldate, pparticipants,  pemployee, pai_value):
    ##Instrucciones para consultar datos de la tabla prueba2
    try:
        insert_datos = '''insert into analisis_sentimientos(file_name,
        upload_date,
        file_size,
        client,
        calldate,
        participants,
        employee,
        ai_value)
        values(:tagfile_name, :tagupload_date, :tagfile_size, :tagclient, :tagcalldate, :tagparticipants,  :tagemployee, :tagai_value)'''
        cur_01.execute(insert_datos,tagfile_name=pfile_name, tagupload_date=pupload_date, tagfile_size=pfile_size, tagclient=pclient, tagcalldate=pcalldate, tagparticipants=pparticipants,  tagemployee=pemployee,tagai_value=pai_value)
    except Exception as err:
        print("Error insertando datos",err)
    else:
        print("Datos insertados correctamente")
        conn.commit()

def analisis_sentimiento_voz(audio_path):

    audio_sin_extension = os.path.splitext(audio_path)[0]

    new_output_path = os.path.join("/content/drive/MyDrive/TrabajoFinal/AudiosCargados/", audio_sin_extension+".csv")

    output_dir_audio = "/content/drive/MyDrive/TrabajoFinal/AudiosCargados/"

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir_audio, exist_ok=True)

    # Inicializar una lista para almacenar los datos
    data_new_audio = []

    # Extraer características del nuevo archivo de audio
    valor = extract_features(audio_path, new_output_path)

    if valor == "Error al ejecutar OpenSMILE":
      return "Error al extraer características del nuevo archivo de audio"

    else:
        with open(new_output_path, 'r') as file:
            lines_new_audio = file.readlines()

        # Encontrar la línea con los datos (después de @data)
        data_start_index = lines.index('@data\n') + 1

        # Extraer los nombres de los atributos y los valores correspondientes
        feature_names_new_audio = []
        feature_values_new_audio = []

        for line_new_audio in lines_new_audio:
          if line_new_audio.startswith('@attribute'):
            parts = line_new_audio.split()
            feature_names_new_audio.append(parts[1])
          elif not line_new_audio.startswith('@') and line_new_audio.strip():
            feature_values_new_audio = line_new_audio.strip().split(',')

        # Crear un diccionario con las características y sus valores
        feature_data_new_audio = {feature_names_new_audio[i]: float(feature_values_new_audio[i]) if feature_values_new_audio[i] != '?' else None for i in range(1, len(feature_values_new_audio))}
        feature_data_new_audio['label'] = emotion

        # Añadir las características a la lista de datos
        data_new_audio.append(feature_data_new_audio)

        # Combinar todos los datos en un DataFrame
        df_new_audio = pd.DataFrame(data_new_audio)

        # Guardar los datos en un archivo CSV
        df_new_audio.to_csv(new_output_path, index=False)

        # Leer las características extraídas
        new_audio_features = pd.read_csv(new_output_path, delimiter=',')
        new_audio_features = new_audio_features.drop(columns=['label', 'class'])

        # Escalar las características del nuevo audio usando el mismo StandardScaler
        new_audio_features = scaler.transform(new_audio_features)  # Apply the same scaling

        # Predecir el sentimiento
        prediction = model_voice.predict(new_audio_features)
        predicted_label = encoder.inverse_transform(np.argmax(prediction, axis=1))

        nivel_satisfaccion_voz = ""

        if predicted_label[0] in ("VeryAngry", "VerySad", "VeryDisgust"):
          nivel_satisfaccion_voz = "Very unsatisfied"
        elif predicted_label[0] in ("Angry", "Sad", "Disgust"):
          nivel_satisfaccion_voz = "Unsatisfied"
        elif predicted_label[0] in ("Neutral"):
          nivel_satisfaccion_voz = "Neutral"
        elif predicted_label[0] in ("Happy"):
          nivel_satisfaccion_voz = "Satisfied"
        elif predicted_label[0] in ("VeryHappy"):
          nivel_satisfaccion_voz = "Very satisfied"


        return nivel_satisfaccion_voz


# Función para escribir el texto del audio
def speech_to_text(audio, company, employee, calldate, participants):
  try:
    model = whisper.load_model("base")
    result = model.transcribe(audio)
    texto = result["text"]
    # Ruta destino del archivo en la carpeta deseada
    dest_path = os.path.join(os.path.basename(audio))
    shutil.move(audio, dest_path)
    AudiosinExtension = os.path.splitext(dest_path)[0]

    file = open(AudiosinExtension+".txt", "w")
    file.write(texto)
    file.close()

    AnalisisSentimientoTexto = analisis_sentimiento_texto(texto, company, employee, calldate, participants,origen="1")
    AnalisisSentimientoVoz = analisis_sentimiento_voz(dest_path)

    return AnalisisSentimientoVoz
  except Exception as e:
    print(f"Error: {e}")
    return f"Se produjo un error: {e}"

def analisis_sentimiento_texto(text, company, employee, calldate, participants,origen):

  # se separa el texto por lineas para hacer la separación
  print(text)
  lineas = text.split('\n')
  print(lineas)
  # Lista para almacenar las líneas del cliente
  lineas_cliente = []

  # Variable para indicar si estamos dentro de un diálogo del cliente
  es_dialogo = False

  # Itera sobre las líneas del archivo
  if origen == "0":
      for linea in lineas:
          if es_dialogo and linea != "":
              lineas_cliente.append(linea.strip())

          # Verifica si la línea actual es dicha por el cliente
          if linea.startswith(employee):
              es_dialogo = True
          elif linea == '':
              es_dialogo = False

  elif origen=="1":
      lineas_cliente=lineas[0].split('.')

  # Imprimir el contenido en bloques de 15 líneas
  bloque_size = 15
  total_lineas = len(lineas_cliente)
  valores = []
  for i in range(0, total_lineas, bloque_size):
    parrafo = ''
    bloque = lineas_cliente[i:i + bloque_size]
    for linea in bloque:
        parrafo = parrafo + ' ' + linea.strip()
    neg, neu, pos = analizar(parrafo)
    valores.append([neg, neu, pos])

  pneg, pneu, ppos = np.mean(valores, axis=0)
  satisfaccion = clasificar_satisfaccion(pneg, pneu, ppos)
  return satisfaccion

def analizar(texto):
  # creación del pipeline cargando el modelo preentrenado
  pipe = pipeline(
      model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
      top_k=None)
  # obtención de la predicción
  evaluacion = pipe(texto)[0]

  # procesamiento de la respuesta del modelo
  if evaluacion[0]['label']=='negative': neg = evaluacion[0]['score']
  elif evaluacion[0]['label']=='positive': pos = evaluacion[0]['score']
  else: neu = evaluacion[0]['score']

  if evaluacion[1]['label']=='negative': neg = evaluacion[1]['score']
  elif evaluacion[1]['label']=='positive': pos = evaluacion[1]['score']
  else: neu = evaluacion[1]['score']

  if evaluacion[2]['label']=='negative': neg = evaluacion[2]['score']
  elif evaluacion[2]['label']=='positive': pos = evaluacion[2]['score']
  else: neu = evaluacion[2]['score']

  return (neg, neu, pos)

# Definir los límites de cada categoría
def clasificar_satisfaccion(negativo, neutro, positivo):
    if positivo >= 0.8:
        return "Very satisfied"
    elif positivo >= 0.4 and positivo > negativo:
        return "Satisfied"
    elif negativo >= 0.8:
        return "Very unsatisfied"
    elif negativo >= 0.4 and negativo > positivo:
        return "Unsatisfied"
    elif neutro >= 0.4:
        return "Neutral"
    else:
        return "Neutral"


styles = {
     "body": {
        "background-color": "white",
        "margin": "0",
        "padding": "0",
        "height": "100vh",
        "display": "flex",
        "justify-content": "center",
        "align-items": "center"
    },
    ".gradio-container": {
        "background-color": "white",
        "width": "100%",
        "height": "100%",
        "display": "flex",
        "justify-content": "center",
        "align-items": "center"
    },
    ".lg.primary.svelte-cmf5ev": {
        "background-color": "#5AAAF0",
        "color": "white",
        "font-size": "16px",
        "border-radius": "125px",
        "margin": "0px 10px"
    },
     ".lg.secondary.svelte-cmf5ev": {
        "background-color": "#B5B7CF",
        "color": "white",
        "font-weight": "bold",
        "font-size": "16px",
        "padding": "10px 20px",
        "border-radius": "125px",
        "margin": "20px 10px"
     },
    ".svelte-1bvc1p0 th": {
        "background-color": "#5AAAF0",
        "color": "white",
        "font-weight": "bold",
        "font-size": "14px",
        "border": "1px solid #ddd",
        "text-align": "center"
    },
    ".svelte-1bvc1p0 td": {
        "background-color": "white",
        "color": "black",
        "border": "1px solid #ddd",
        "text-align": "center"
    },
     ".block.svelte-12cmxck": {
        "background-color": "white",
        "color": "black",
        "text-align": "center",
        "border": "1px solid white"
    },
     ".svelte-1b6s6s": {
         "display": "inline-block",
         "background-color": "white",
         "color": "black",
         "border": "1px solid black"
    },
     ".svelte-j5bxrl": {
         "color": "#B5B7CF",
         "background-color": "#B5B7CF",
         "border": "1px solid black"
    },
     ".svelte-j5bxrl svg": {
         "color": "#B5B7CF",
         "margin-right": "10px"
    },
     ".wrap": {
        "color": "black"
    },
     ".or": {
         "color": "black"
    },
     ".svelte-1gfkn6j": {
         "color": "black"
    },
     ".svelte-sa48pu.stretch":{
         "background-color": "white",
         "border": "1px solid white",
         "align-items": "center",
         "justify-content": "center"
    },
     ".svelte-iyf88w": {
         "border": "1px solid white"
    },
     ".svelte-1f354aw textarea": {
         "border": "1px solid black",
         "background-color": "#B5B7CF",
         "font-weight": "bold",
         "color": "black"
    },
     ".svelte-vt1mxs": {
         "padding": "0"
    },
     "#component-231": {
         "margin-top": "10px"
    },
     ".svelte-vt1mxs gap": {
         "padding": "0"
     }
}


css_styles = "\n".join([f"{selector} {{{' '.join([f'{prop}: {value};' for prop, value in props.items()])}}}" for selector, props in styles.items()])

def on_upload_submit(file, company, employee, calldate, participants):
    # Verificar que se haya subido un archivo
    if file:
        # Verificar si el archivo es un texto
        if ((file.name).find('.txt') != -1):
            with open(file.name, 'r') as f:
                content = f.read()
            valor = analisis_sentimiento_texto(content, company, employee, calldate, participants,origen="0")
            write_db(pfile_name=file.name, pupload_date=datetime.now().date(), pfile_size=100, pclient=company, pcalldate=calldate, pparticipants=int(participants), pemployee=employee, pai_value=valor)
            return valor
        # Verificar si el archivo es un audio
        elif ((file.name).find('.wav') != -1):
            audio_a_texto = speech_to_text(file, company, employee, calldate, participants)
            write_db(pfile_name=file.name, pupload_date=datetime.now().date(), pfile_size=100, pclient=company, pcalldate=calldate, pparticipants=int(participants), pemployee=employee, pai_value=audio_a_texto)
            return audio_a_texto
        else:
            return "El archivo cargado no tiene formato .txt o .wav."
    else:
        return "No se ha seleccionado ningún archivo."

def query_to_DB(param_client):
    try:
        select_datos = '''select upload_date,
        calldate,
        employee,
        client,
        participants,
        ai_value
        from analisis_sentimientos
        where client=:tagclient'''
        cur_01.execute(select_datos, tagclient=param_client)
        rows = cur_01.fetchall()
        for row in rows:
            print(row)
        return rows
    except Exception as err:
        print(f"Error:{err}")
        return f"Se produjo un error: {err}"

with gr.Blocks(css=css_styles, theme=gr.themes.Monochrome()) as webpage:
    markdown_text = """
    <div style='text-align: center; font-size: 36px; color: Black; font-weight: bold'>
    Calls Sentiment Analysis
    </div>
    """
    gr.Markdown(markdown_text)

    # Crear botón cargar archivo, buscar y tabla de resultados
    new_text_file_button = gr.Button("Upload .txt or .wav file", variant="primary")
    new_query_button = gr.Button("Search call by Client", variant="primary")

    # Crear los componentes para el archivo de texto
    upload_file = gr.Group(visible=False)
    query_db = gr.Group(visible=False)

    with upload_file:
        with gr.Row():
            file_input = gr.File(label="Select a File", file_count="single", file_types=[".txt", ".wav" ], interactive=True)

            with gr.Row():
                company_input = gr.Textbox(label="Client")
                employee_input = gr.Textbox(label="Employee")
            with gr.Row():
                calldate_input = gr.Textbox(label="Call date (YYYY-MM-DD)")
                participants_input = gr.Textbox(label="Participants")
            with gr.Row():
                close_button = gr.Button("Close", variant="secondary")
                upload_text_button = gr.Button("Upload", variant="primary")


            upload_text_button.click(on_upload_submit, inputs=[file_input, company_input, employee_input, calldate_input, participants_input], outputs=[gr.Textbox(label="Sentiment Analysis", lines=10)])
            close_button.click(lambda: gr.update(visible=False), inputs=[], outputs=[upload_file])

    with query_db:
        with gr.Row():
            cliente_query = gr.Textbox(label="Client")
            query_client_button = gr.Button("Search", variant="primary")
        lista_datos_db = gr.Dataframe(headers=["Date uploaded", "Call Date", "Employee", "Client", "Participants", "AI Value"],
                                          datatype=["date", "date", "str", "str", "number", "str"],
                                          row_count=1)
        query_client_button.click(query_to_DB, inputs=cliente_query, outputs=lista_datos_db)

    new_query_button.click(lambda: gr.update(visible=True), inputs=[], outputs=[query_db])
    new_text_file_button.click(lambda: gr.update(visible=True), inputs=[], outputs=[upload_file])

webpage.launch()

23.5.0.24.5
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4d014796b0c12616b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
